In [1]:
import numpy as np
import pointCollection as pc
import matplotlib.pyplot as plt
from make_sim_along_track import make_sim_tracks, apply_errors, apply_shifts
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
xy0 = [-160000, -2170000]


In [ ]:
# make tracks, shift by RGT pointing error
xy0 = [-160000, -2170000]
D=make_sim_tracks(EPSG=3413, x_0=xy0[0], y_0=xy0[1], product='ATL06',
                  t_unit='year')
apply_shifts(
    D,
    ['rgt','cycle'], 10, update_yatc=True);


In [ ]:
def z_plane(D, mean_slope=0.1, xy0=[0,0] ):
    return (D.x-xy0[0])*mean_slope
z_fn = z_plane
z_args = {'mean_slope':0.1, 'xy0':xy0}
sigma={'geoloc':3.5,'r':0.03,'z_corr':0.03,'z_uncorr':0.15/np.sqrt(57)}
data=apply_errors(D, z_fn = z_plane, z_args=z_args, sigma=sigma)

In [ ]:
plt.figure()
plt.colorbar(
    plt.scatter(data.x[::5], data.y[::5],2, 
                c=data.z[::5] - z_plane(data[::5], **z_args), cmap='RdBu'))

In [ ]:
plt.figure()
plt.hist(data.z[::5] - z_plane(data[::5], **z_args), 50);

In [ ]:
data.get_xy(EPSG=3413)

In [ ]:
from LSsurf.smooth_fit import smooth_fit

# ATL14 parameters
E_d3zdx2dt=0.0001
E_d2z0dx2=0.06
E_d2zdt2=5000

cycle_range=[np.min(data.cycle), np.max(data.cycle)]
bias_params=None
data.assign({'sigma':np.zeros_like(data.z)+0.1})

data_nobias=data.copy()

ctr={'x':xy0[0],'y':xy0[1],'t':np.mean(cycle_range)*0.25}
W={'x':2.e4,'y':2.e4,'t':np.diff(cycle_range)*0.25}
spacing={'z0':100, 'dz':500, 'dt':0.25}

data_gap_scale=2500
E_RMS={'d2z0_dx2':E_d2z0dx2, 'dz0_dx':E_d2z0dx2*data_gap_scale, 'd3z_dx2dt':E_d3zdx2dt, 'd2z_dxdt':E_d3zdx2dt*data_gap_scale,  'd2z_dt2':E_d2zdt2}

# run the fit with no bias params
S_nobias=smooth_fit(data=data_nobias[::5], ctr=ctr, W=W, spacing=spacing, E_RMS=E_RMS,
             reference_epoch=5, compute_E=False,
             max_iterations=1,
             bias_params=None,
             VERBOSE=True, dzdt_lags=[1])

bias_params=['cycle','rgt']
data.assign({'sigma_corr':np.zeros_like(data.z)+((z_args['mean_slope']*sigma['geoloc'])**2+0.03**2)**0.5})

# run the fit with bias params
S_bias=smooth_fit(data=data[::5], ctr=ctr, W=W, spacing=spacing, E_RMS=E_RMS,
             reference_epoch=5, compute_E=False,
             max_iterations=1,
             bias_params=bias_params,
             VERBOSE=True, dzdt_lags=[1])


In [ ]:
bias_params=['cycle','rgt']
data.assign({'sigma_corr':np.zeros_like(data.z)+((z_args['mean_slope']*sigma['geoloc'])**2+0.03**2)**0.5})

# run the fit with bias params
S_bias=smooth_fit(data=data[::5], ctr=ctr, W=W, spacing=spacing, E_RMS=E_RMS,
             reference_epoch=5, compute_E=False,
             max_iterations=1,
             bias_params=bias_params,
             VERBOSE=True, dzdt_lags=[1])


In [ ]:

hf, hax=plt.subplots(1, 2)
hax[0].imshow(np.std(S_nobias['m']['dz'].dz, axis=2), clim=[0, 0.5])
hax[1].imshow(np.std(S_bias['m']['dz'].dz, axis=2), clim=[0, 0.5])
